# QUEENS walckthrough: library usage

This notebook illustrates how to use the library methods from QUEENS to process DUKES data, query the machine-readable tables and export them to file.

To run the code below, make sure you do `pip install queens` in your environment.

## Setting up configuration paths.

By default, `queens` will create a user directory in your local machine where it will save all the resources. On Windows this is typically in `C:/Users/username/AppData/Local/queens/`.

Some of these files can be saved in alternative custom locations.

In [ ]:
import queens as q

# view the current paths with the following command
print(q.settings.DB_PATH)
print(q.settings.EXPORT_DIR)

# You can customise the default db_path and the export_path using
# your_db_file_path = "mydir/queens_db.db"
# your_export_dir_path = "mydir/exports/"
# q.set_config(db_path=your_db_file_path, export_path=your_export_dir_path)

# optionally, you can make the methods print log messages
# they will also be saved to a file in the user directory
q.setup_logging(level="info")

## Ingesting data

QUEENS will read the data directly from the GOV.UK urls provided in the `user_dir/queens/urls.json` file. These links point at the most recent version of each chapter of the data collection, so that the most recent publicly available datasets are ingested.

You can ingest a single table, a selection of them, or the full data collection.

Whether a table can be ingested or not, depends on the presence of a corresponding mapping template (in `user_dir/queens/templates/` and/or an entry in the `user_dir/queens/etl_config.json` configuration file that tells the engine how to reshape that particular table.

With the package installer, most of the DUKES tables can be ingested without further work. However the user can easily tweak the existing templates or add new ones. Refer to the documentation to make changes.

In [5]:
# ingest a selection of tables
q.ingest(
    data_collection="dukes",
    tables=["1.1", "6.1", "J.1", "7.4"]
)

# ingest all tables in DUKES
# q.ingest("dukes")

2025-08-24 17:25:02,646 - INFO - Creating _ingest_log.
2025-08-24 17:25:02,864 - INFO - Creating _metadata.
2025-08-24 17:25:03,033 - INFO - Creating dukes_raw.
2025-08-24 17:25:03,151 - INFO - Processing dukes table 1.1.
2025-08-24 17:25:10,027 - INFO - Ingesting subtable 1.1.
2025-08-24 17:25:10,616 - INFO - ETL successful for table 1.1
2025-08-24 17:25:10,616 - INFO - Processing dukes table 6.1.
2025-08-24 17:25:16,757 - INFO - Ingesting subtable 6.1.
2025-08-24 17:25:17,437 - INFO - ETL successful for table 6.1
2025-08-24 17:25:17,439 - INFO - Processing dukes table J.1.
2025-08-24 17:25:21,422 - INFO - Ingesting subtable J.1.
2025-08-24 17:25:21,951 - INFO - ETL successful for table J.1
2025-08-24 17:25:21,951 - INFO - Processing dukes table 7.4.
2025-08-24 17:25:25,601 - INFO - Ingesting subtable 7.4.A.
2025-08-24 17:25:25,931 - INFO - Ingesting subtable 7.4.B.
2025-08-24 17:25:26,287 - INFO - ETL successful for table 7.4
2025-08-24 17:25:26,288 - INFO - ETL run completed success

## Inspecting data versions

Every ingested table is logged into a table that registers the ingest time and the source. This is used to version ingested data and to distinguish between various versions of the same table.

In the example below, two versions of table 1.1 are present in the databse.

In [9]:
# ingest specific tables
q.versions(
    data_collection="dukes"
    # table_name is optional
)


,Ingest date,Ingest time
Table number,,
1.1,2025-08-24,17:28:07.430015
1.1,2025-08-24,17:25:03.151712
6.1,2025-08-24,17:25:03.151712
7.4.A,2025-08-24,17:25:03.151712
7.4.B,2025-08-24,17:25:03.151712
J.1,2025-08-24,17:25:03.151712


## Staging

In order to analyse data, we need the most current version to be the only available. The `stage` method will extract the most recent version of each table_name and store it in a separate, snapshot table.

If the use wants to stage a previously ingested version, they can specify the `as_of_date` argument, which will restrict the view to ingested data older than the passed date.

In [10]:
# stage the ost recent as of today
q.stage("dukes")

# stage past ingested data
# q.stage("dukes", as_of_date="2025-05-26")

2025-08-24 17:39:17,440 - INFO - Data for dukes successfully staged in prod. 
This is a snapshot as of today


## Metadata

In the output of `q.versions` above, note that there are two entries for table 7.4: 7.4.A and 7.4.B. This happens when a high level table has multiple sheets that relate to different data. Other tables, where the same content is spread across sheets, are ingested as a single entry (i.e. tables 1.1 and 6.1). 

Due to the varied nature of the tables even within the same heading, the relevant column names can vary a lot. To see which columns are "queryable" for each table_name, use the `metadata` method

This method will display the list of accessible columns (i.e. not wholly null), the number of non-nulls, the number of unique values and the data type (as a SQLite datatype).

In [11]:
# view queryable columns for table 7.4.A
q.metadata("dukes", table_name="7.4.A")

,data_collection,table_name,column_name,n_non_nulls,n_unique,dtype
0,dukes,7.4.A,item,4960,8,TEXT
1,dukes,7.4.A,label,4960,27,TEXT
2,dukes,7.4.A,row,4960,160,INTEGER
3,dukes,7.4.A,sector,4960,3,TEXT
4,dukes,7.4.A,subsector,4960,19,TEXT
5,dukes,7.4.A,unit,4960,3,TEXT
6,dukes,7.4.A,value,4960,4307,REAL
7,dukes,7.4.A,year,4960,31,INTEGER


## Querying data

Once data has been staged, it can be queried at the user discretion. The `query` method provides a flexible interface for querying and filtering data.

Filters must be submitted as dictionaries and are subject to strict validation rukes. Please see `docs/filters.md` for generat instructions. 

The example below is akin to the following SQL query:
```
SELECT *
FROM dukes_prod
WHERE
    table_name = '6.1'
    AND year >= 2010
    AND (fuel = 'Landfill gas' OR fuel LIKE '%waste%')
```

Note: the match on text fields is case insensitive.

In [21]:
# define the filtering
filters = {
    "year": {"gte": 2020},
    "$or": [
         {"fuel": "Landfill gas"},
        {"fuel": {"like": "%waste%"}}
    ]
}

# get the data as a dataframe
df = q.query(
    data_collection="dukes",
    table_name="6.1",
    filters=filters
)

df.head(10)

,table_description,table_name,row,label,year,group,subgroup,category,item,fuel,unit,value
0,Renewables and waste: commodity balances (DUKE...,6.1,0,Production,2024,Supply,Production,Production,Production,Waste wood,ktoe,403.34
1,Renewables and waste: commodity balances (DUKE...,6.1,1,Other sources,2024,None,None,None,None,Waste wood,None,0.00
2,Renewables and waste: commodity balances (DUKE...,6.1,2,Imports,2024,Supply,Imports,Imports,Imports,Waste wood,ktoe,56.99
3,Renewables and waste: commodity balances (DUKE...,6.1,3,Exports,2024,Supply,Exports,Exports,Exports,Waste wood,ktoe,-162.64
4,Renewables and waste: commodity balances (DUKE...,6.1,4,Marine bunkers,2024,Supply,Marine bunkers,Marine bunkers,Marine bunkers,Waste wood,ktoe,0.00
5,Renewables and waste: commodity balances (DUKE...,6.1,5,Stock change [note 1],2024,Supply,Stock change,Stock change,Stock change,Waste wood,ktoe,0.00
6,Renewables and waste: commodity balances (DUKE...,6.1,6,Transfers,2024,Supply,Transfers,Transfers,Transfers,Waste wood,ktoe,0.00
7,Renewables and waste: commodity balances (DUKE...,6.1,7,Total supply,2024,Supply,Supply,Supply,Supply,Waste wood,ktoe,297.69
8,Renewables and waste: commodity balances (DUKE...,6.1,8,Statistical difference [note 2],2024,Statistical difference,Statistical difference,Statistical difference,Statistical difference,Waste wood,ktoe,0.00
9,Renewables and waste: commodity balances (DUKE...,6.1,9,Total demand,2024,Demand,Demand,Demand,Demand,Waste wood,ktoe,297.69


## Exporting to file

QUEENS is also able to export the staged data as a flat file. Supported formats are CSV, XLSX and Parquet. 

The `export` method gives the choice of specifying a custom output path that overrides the default location (specified above). 

File names are always suffixed with a timestamp. By default, the method exports one file per table name; it is possible to export an entire data collection, either as a set of separate files or as a single, bulk file (`bulk=True`). 

For Excel exports, the bulk export feature generates a single workbook with one `table_name` per sheet.

In [ ]:
# export table to CSV
q.export(
    data_collection="dukes",
    table_name="7.4.A",
    file_type="csv"
)

# export the whole of DUKES as an Excel file
q.export(
    data_collection"dukes,
    file_type="xslx",
    bulk_export=True
)